In [5]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu129

Looking in indexes: https://download.pytorch.org/whl/cu129
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.2/581.2 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import re
import pandas as pd
import numpy as np

In [7]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [8]:
tokens = tokenizer.encode("I hate it absolutely worst",return_tensors='pt')
tokens

tensor([[  101,   151, 39487, 10197, 35925, 10563, 43060,   102]])

In [9]:
tokenizer.decode(tokens[0])

'[CLS] i hate it absolutely worst [SEP]'

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abhi8923shriv/sentiment-analysis-dataset")

print("Path to dataset files:", path)

100%|██████████| 54.4M/54.4M [00:00<00:00, 121MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/abhi8923shriv/sentiment-analysis-dataset/versions/9


In [10]:
import os
print(os.listdir(path))

['testdata.manual.2009.06.14.csv', 'test.csv', 'training.1600000.processed.noemoticon.csv', 'train.csv']


In [27]:
data = pd.read_csv(
    path + '/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')
data

,polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1048567,4,1960186342,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Madelinedugganx,My GrandMa is making Dinenr with my Mum
1048568,4,1960186409,Fri May 29 07:33:43 PDT 2009,NO_QUERY,OffRoad_Dude,Mid-morning snack time... A bowl of cheese noo...
1048569,4,1960186429,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Falchion,@ShaDeLa same here say it like from the Termi...
1048570,4,1960186445,Fri May 29 07:33:44 PDT 2009,NO_QUERY,jonasobsessedx,@DestinyHope92 im great thaanks wbuu?


In [28]:
sentiment = data.iloc[0:100]
sentiment

,polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
95,0,1467836859,Mon Apr 06 22:26:33 PDT 2009,NO_QUERY,willy_chaz,A bad nite for the favorite teams: Astros and ...
96,0,1467836873,Mon Apr 06 22:26:33 PDT 2009,NO_QUERY,LeakySpoon,Body Of Missing Northern Calif. Girl Found: P...
97,0,1467837189,Mon Apr 06 22:26:38 PDT 2009,NO_QUERY,cityrat59,@mangaaa I hope they will increase the capacit...
98,0,1467837384,Mon Apr 06 22:26:42 PDT 2009,NO_QUERY,kenandise,Behind on my classes for work


In [29]:
sentiment.columns

Index(['polarity of tweet ', 'id of the tweet', 'date of the tweet', 'query',
       'user', 'text of the tweet '],
      dtype='object')

In [35]:
sentiment.columns = sentiment.columns.str.strip()
sentiments = sentiment[['text of the tweet']].copy()
sentiments['text of the tweet'] = sentiments['text of the tweet'].astype(str).fillna(" ")
from sklearn.utils import shuffle

sentiments = shuffle(sentiments)
sentiments = sentiments.reset_index(drop=True)
sentiments

,text of the tweet
0,"@andywana Not sure what they are, only that th..."
1,is strangely sad about LiLo and SamRo breaking...
2,wednesday my b-day! don't know what 2 do!!
3,Put vacation photos online a few yrs ago. PC c...
4,"one of my friend called me, and asked to meet ..."
...,...
95,Just going to cry myself to sleep after watchi...
96,"@octolinz16 It it counts, idk why I did either..."
97,My tummy hurts. I wonder if the hypnosis has ...
98,I have a sad feeling that Dallas is not going ...


In [36]:
def sentiment_score(tweet):
  tokens = tokenizer.encode(tweet,return_tensors='pt')
  results = model(tokens)
  sentiment = int(torch.argmax(results.logits))+1
  if sentiment in [1,2]:
    return 'positive'
  elif sentiment in [4,5]:
    return 'negative'
  else:
    return 'neutral'

In [39]:
sentiments['label'] = sentiments['text of the tweet'].apply(lambda x: sentiment_score(x[:512]))
sentiments

,text of the tweet,label
0,"@andywana Not sure what they are, only that th...",positive
1,is strangely sad about LiLo and SamRo breaking...,positive
2,wednesday my b-day! don't know what 2 do!!,negative
3,Put vacation photos online a few yrs ago. PC c...,positive
4,"one of my friend called me, and asked to meet ...",positive
...,...,...
95,Just going to cry myself to sleep after watchi...,negative
96,"@octolinz16 It it counts, idk why I did either...",neutral
97,My tummy hurts. I wonder if the hypnosis has ...,positive
98,I have a sad feeling that Dallas is not going ...,neutral
